In [1]:
import os

from rdkit import Chem # safe import before ccdc imports
from torch_geometric.loader import DataLoader
from torch.utils.data import ConcatDataset

from conf_ensemble_dataset_in_memory import ConfEnsembleDataset
from litschnet import LitSchNet
from rmsd_predictor_evaluator import RMSDPredictorEvaluator
from tqdm import tqdm

# Data preparation

In [3]:
data_dir = 'data/'

In [4]:
# run once to preprocess datasets and generate chunks
# dataset = ConfEnsembleDataset()
# dataset = ConfEnsembleDataset(dataset='platinum') # 16G

In [5]:
platinum_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('platinum')]
platinum_n_chunks = len(platinum_chunks)

In [6]:
platinum_datasets = []
for chunk_number in tqdm(range(platinum_n_chunks)) :
    dataset = ConfEnsembleDataset(dataset='platinum', loaded_chunk=chunk_number)
    platinum_datasets.append(dataset)
platinum_dataset = ConcatDataset(platinum_datasets)

100%|██████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.13s/it]


In [7]:
pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
pdbbind_n_chunks = len(pdbbind_chunks)

In [8]:
splits = ['random', 'scaffold']
tasks = ['all', 'easy', 'hard']

In [9]:
%%time

for split in splits :

    for iteration in range(5) :

        with open(os.path.join(data_dir, f'{split}_splits', f'test_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            test_smiles = f.readlines()
            test_smiles = [smiles.strip() for smiles in test_smiles]

        test_datasets = []

        for chunk_number in tqdm(range(pdbbind_n_chunks)) :

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=test_smiles)
            test_datasets.append(dataset)

        test_dataset = ConcatDataset(test_datasets)

        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

        experiment_name = f'{split}_split_{iteration}_v2'
        if experiment_name in os.listdir('lightning_logs') :
            checkpoint_name = os.listdir(os.path.join('lightning_logs', experiment_name, 'checkpoints'))[0]
            checkpoint_path = os.path.join('lightning_logs', experiment_name, 'checkpoints', checkpoint_name)
            litschnet = LitSchNet.load_from_checkpoint(checkpoint_path=checkpoint_path)

            evaluation_name = experiment_name + '_pdbbind'
            evaluator = RMSDPredictorEvaluator(model=litschnet, evaluation_name=evaluation_name)
            evaluator.evaluate(test_dataset)
            for task in tasks :
                evaluator.evaluation_report(task=task)

            evaluation_name = experiment_name + '_platinum'
            evaluator = RMSDPredictorEvaluator(model=litschnet, evaluation_name=evaluation_name)
            evaluator.evaluate(platinum_dataset)
            for task in tasks :
                evaluator.evaluation_report(task=task)

100%|██████████████████████████████████████████████████████████████████| 3/3 [01:06<00:00, 22.08s/it]


Grouping data by smiles


  0%|                                                                       | 0/1085 [00:00<?, ?it/s]

Starting evaluation


100%|████████████████████████████████████████████████████████████| 1085/1085 [00:21<00:00, 51.25it/s]


...

Grouping data by smiles


  0%|                                                               | 2/4548 [00:00<04:18, 17.57it/s]

Starting evaluation


100%|████████████████████████████████████████████████████████████| 4548/4548 [01:24<00:00, 53.65it/s]


CPU times: user 3h 41min 8s, sys: 16.8 s, total: 3h 41min 24s
Wall time: 35min 40s
